In [16]:
AMECO = pd.read_csv("data/AMECO.csv", sep=";")

def get_ameco(code,ISO):
    code = str(code)
    ISO = str(ISO)
    identifier = ISO + "." + code
    return AMECO[AMECO["CODE"] == identifier]

df_jst = pd.read_stata("http://macrohistory.net/JST/JSTdatasetR4.dta", columns =["year", "iso", "eq_dp"])
df_jst['year'] = df_jst['year'].astype(int)

def get_jst(iso):
    return df_jst[(df_jst["iso"]==iso) & df_jst['year'].isin(range(1960,2015+1))].loc[:,"eq_dp"].tolist()


codes = ['1.0.0.0.NPTD','3.1.0.0.PIGT','3.0.0.0.ZCPIN','1.0.0.0.NETD','3.0.0.0.ZVGDF','1.0.0.0.ALCD0','1.1.0.0.OIGT','1.0.0.0.OKND','1.0.0.0.AKNDV','1.3.0.0.ISRC']
countries = ['BEL','CAN','DNK','FIN','FRA','DEU','ITA','JPN','NLD','NOR','PRT','CHE','GBR','USA']

In [17]:
parts = []
for ISO in countries:
    
    #Create dfx with all series for country
    dfx = pd.DataFrame()
    for code in codes:
        data = get_ameco(code,ISO).loc[:,"1960":"2015"]
        data = data.transpose()
        data.columns = [code]
        dfx[code] = data[code]

    dfx["eq_dp"] = get_jst(ISO)
    dfx["ISO"] = ISO
    dfx["year"] = dfx.index.tolist()
    dfx.reset_index(drop=True)
    #End
    parts.append(dfx)
    
df_final = pd.concat(parts, ignore_index = True)
df_final.to_csv("data/data.csv" ,sep=";", index=False)

In [18]:
df_final

,1.0.0.0.NPTD,3.1.0.0.PIGT,3.0.0.0.ZCPIN,1.0.0.0.NETD,3.0.0.0.ZVGDF,1.0.0.0.ALCD0,1.1.0.0.OIGT,1.0.0.0.OKND,1.0.0.0.AKNDV,1.3.0.0.ISRC,eq_dp,ISO,year
0,9153.489,16.174873,15.550910,3529.071,46.895165,55.869410,22.3550,289.499,3.000000,NaN,0.047511,BEL,1960
1,9183.948,16.345739,15.701149,3553.023,48.503186,54.719370,25.1188,297.488,2.936713,2.123425,0.041592,BEL,1961
2,9220.578,17.007553,15.927608,3598.786,50.019058,55.673472,26.5958,306.675,2.878273,2.308766,0.037479,BEL,1962
3,9289.770,17.736773,16.269129,3603.373,51.620532,56.043757,26.6352,315.203,2.833520,-0.093245,0.034060,BEL,1963
4,9378.113,18.733620,16.947040,3625.592,54.224142,55.362434,30.5413,326.797,2.746898,0.644751,0.034231,BEL,1964
5,9463.667,19.577586,17.636677,3621.413,55.396971,55.628684,31.8007,339.071,2.751399,0.233437,0.037589,BEL,1965
6,9527.807,20.186075,18.371752,3627.019,56.188086,56.450595,33.9714,352.652,2.775372,1.413524,0.041322,BEL,1966
7,9580.991,21.009714,18.897225,3613.871,57.603342,56.393579,34.9587,366.593,2.777629,2.759871,0.043135,BEL,1967
8,9618.756,21.357165,19.420498,3605.411,59.310540,55.968139,34.5189,378.971,2.756407,1.613972,0.040516,BEL,1968
9,9646.032,22.272348,20.146779,3656.984,61.868723,55.522421,36.3598,391.974,2.673891,4.363217,0.040967,BEL,1969
